# 01) Single-omics EDA

**실습 목표**: mRNA·Clinical·Methylation 데이터의 탐색적 분석 및 DMP 분석

## 라이브러리 및 환경 설정

In [ ]:
import numpy as np, pandas as pd, matplotlib.pyplot as plt, seaborn as sns
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from scipy import stats
import warnings
warnings.filterwarnings('ignore')

import plot_config
plot_config.setup()
np.random.seed(42)

## 데이터 로드

In [ ]:
# 파일 경로
MRNA = "input/mrna_matched.txt"
CLIN = "input/clinical_matched.tsv"
METH = "input/methylation_matched.txt"             

# 임상 컬럼명
SAMPLE_COL   = "Sample ID"
STATUS_COL   = "Overall Survival Status"      # "1:DECEASED","0:LIVING"
DURATION_COL = "Overall Survival (Months)"
SUBTYPE_COL  = "Subtype"
STAGE_COL    = "Neoplasm Disease Stage American Joint Committee on Cancer Code"
AGE_COL      = "Diagnosis Age"

## 데이터 로드 및 공통 샘플 정렬

In [ ]:
mrna = pd.read_csv(MRNA, sep="\t", index_col=0)
clinical = pd.read_csv(CLIN, sep="\t")
methyl = pd.read_csv(METH, sep="\t", index_col=0)

# 공통 샘플 정렬
common = sorted(set(mrna.columns) & set(clinical[SAMPLE_COL]) & set(methyl.columns))
mrna = mrna[common]
clinical = clinical[clinical[SAMPLE_COL].isin(common)].reset_index(drop=True)
methyl = methyl[common]
mrna_num = mrna.select_dtypes(include=[np.number])

print(f"mRNA: {mrna.shape} | clinical: {clinical.shape} | methyl: {methyl.shape}")

## 임상 데이터 EDA

In [ ]:
fig, axes = plt.subplots(1, 3, figsize=(16, 4))

clinical[STATUS_COL].value_counts().plot(kind="pie", autopct="%1.0f%%", ax=axes[0], title="Survival Status")

sns.countplot(x=SUBTYPE_COL, data=clinical, ax=axes[1])
axes[1].set_title("Subtype Distribution")
axes[1].tick_params(axis="x", rotation=30)

clinical[AGE_COL].plot(kind="hist", bins=20, edgecolor="black", ax=axes[2], title="Age Distribution")

plt.tight_layout()
plt.show()

## mRNA 발현 데이터 EDA

In [ ]:
fig, axes = plt.subplots(2, 2, figsize=(16, 12))
fig.suptitle("mRNA Data Analysis", fontsize=16)

# 1. 바이올린 플롯 (상위 5개 유전자)
sns.violinplot(data=mrna_num.iloc[:5].T, ax=axes[0, 0])
axes[0, 0].set_title("Expression Distribution (Top 5 Genes)")
axes[0, 0].tick_params(axis='x', rotation=45)

# 2. 밀도 플롯
g = mrna_num.index[0]
sns.kdeplot(mrna_num.loc[g], fill=True, ax=axes[0, 1])
axes[0, 1].set_title(f"Expression Density: {g}")

# 3. 샘플 간 상관관계
sns.heatmap(mrna_num.corr(), cmap="vlag", ax=axes[1, 0])
axes[1, 0].set_title("Sample Correlation Heatmap")

# 4. PCA
X = StandardScaler().fit_transform(mrna_num.T.fillna(0))
pca = PCA(n_components=2).fit(X)
pcs = pca.transform(X)
axes[1, 1].scatter(pcs[:, 0], pcs[:, 1])
axes[1, 1].set_title(f"PCA Plot (PC1 explains {pca.explained_variance_ratio_[0]:.1%})")
axes[1, 1].set_xlabel("PC1"); axes[1, 1].set_ylabel("PC2")

plt.tight_layout(rect=[0, 0, 1, 0.96])
plt.show()

### Vibe Coding 미션 A1: mRNA 발현 분석 심화
**미션**: mRNA 발현 데이터 시각화를 개선해보세요!

**결과물**: 색상이 개선된 바이올린 플롯 + 통계선이 포함된 밀도 플롯 + 아형별 PCA 산점도

**도전 과제**:
1. 바이올린 플롯 색상을 'husl' 팔레트로 변경하기
2. 밀도 플롯에 평균과 중앙값 선 추가하기  
3. PCA 산점도에 암 아형별 색상 적용하기

In [ ]:
# 미션 A1 코드를 여기에 작성하세요!

### Vibe Coding 미션 A2: 고급 유전자 발현 분석
**미션**: 분산이 큰 유전자들만 선별하여 클러스터맵을 만들어보세요!

**결과물**: Top 30 고분산 유전자의 클러스터맵 (아형별 컬러바 포함)

**도전 과제**:
1. 분산이 큰 상위 **30개 유전자** 선별하기
2. 클러스터맵에 아형별 **컬러바** 추가하기
3. (도전) 범례 위치 지정하기

In [ ]:
# 미션 A2 코드를 여기에 작성하세요!

## 메틸화 데이터 EDA

In [ ]:
# 숫자형 변환 및 전처리
meth = methyl.apply(pd.to_numeric, errors="coerce").groupby(level=0).mean()
min_valid = max(3, int(0.2 * meth.shape[1]))
meth = meth[meth.notna().sum(axis=1) >= min_valid].clip(0, 1)
print(f"메틸화 전처리 완료: {meth.shape}")

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(16, 6))
fig.suptitle("Methylation Data Analysis", fontsize=16)

# 1. 베타값 분포
axes[0].hist(meth.iloc[:, 0].dropna().clip(0, 1), bins=40, edgecolor="black", alpha=0.7)
axes[0].set_title("Beta Value Distribution (First Sample)")
axes[0].set_xlabel("Beta Value"); axes[0].set_ylabel("Count")

# 2. 상위 변이 프로브 히트맵
var = meth.var(axis=1, skipna=True).sort_values(ascending=False)
top = var.head(30).index
subset = meth.loc[top, meth.columns[:30]]
sns.heatmap(subset, cmap="RdYlBu_r", center=0.5, cbar_kws={'label': 'Beta Value'}, ax=axes[1])
axes[1].set_title("Top Variable Methylation Probes")
axes[1].tick_params(axis='x', rotation=45)

plt.tight_layout(rect=[0, 0, 1, 0.96])
plt.show()

### Vibe Coding 미션 B1: 메틸화 분포 시각화 개선
**미션**: 메틸화 베타값 분포를 더 자세히 분석해보세요!

**결과물**: 5개 샘플의 베타값 히스토그램 + KDE 비교 서브플롯

**도전 과제**:
1. 히스토그램 bin 개수를 **50개**로 설정하기
2. 처음 5개 샘플의 분포를 **서브플롯**으로 비교하기
3. KDE 곡선을 히스토그램에 추가하기

In [ ]:
# 미션 B1 코드를 여기에 작성하세요!


### Vibe Coding 미션 B2: 프로브별 메틸화 비교 분석
**미션**: 상위 변이 프로브들의 메틸화 패턴을 박스플롯으로 비교해보세요!

**결과물**: 20개 고분산 프로브의 베타값 박스플롯

**도전 과제**:
1. 분산이 큰 **20개 프로브** 선별하기
2. 각 프로브별 베타값 분포를 **박스플롯**으로 시각화하기
3. (도전) 아웃라이어가 있는 프로브 식별하고 표시하기

In [ ]:
# 미션 B2 코드를 여기에 작성하세요!

## DMP (Differential Methylated Probe) 분석

암 아형간 메틸화 차이가 큰 프로브를 Welch t-test + FDR 보정으로 식별 → Volcano plot

In [ ]:
diff_thr = 0.10   # |Δβ| 컷오프
fdr_thr  = 0.05   # FDR 컷오프

# 공통 샘플 정렬
clinical_sub = clinical[[SAMPLE_COL, SUBTYPE_COL]].dropna()
common = methyl.columns.intersection(clinical_sub[SAMPLE_COL])
methyl_use   = methyl.loc[:, common]
clinical_use = clinical_sub[clinical_sub[SAMPLE_COL].isin(common)]

# 비교할 두 subtype 자동 선택 (샘플 수 상위 2개)
vc = clinical_use[SUBTYPE_COL].value_counts()
subtype_a, subtype_b = vc.index[:2].tolist()

group_A = clinical_use.loc[clinical_use[SUBTYPE_COL].eq(subtype_a), SAMPLE_COL].tolist()
group_B = clinical_use.loc[clinical_use[SUBTYPE_COL].eq(subtype_b), SAMPLE_COL].tolist()
print(f"비교: {subtype_a} (n={len(group_A)}) vs {subtype_b} (n={len(group_B)})")

In [ ]:
# Δβ 계산 + Welch t-test + BH FDR 보정
mean_A = methyl_use[group_A].mean(axis=1)
mean_B = methyl_use[group_B].mean(axis=1)
diff   = mean_A - mean_B

t_stat, p_val = stats.ttest_ind(methyl_use[group_A].T, methyl_use[group_B].T,
                                equal_var=False, nan_policy="omit")
p_val = np.asarray(p_val, float)
p_val = np.where(np.ndim(p_val)==0, np.full(methyl_use.shape[0], np.nan), p_val)

# Benjamini-Hochberg FDR
order  = np.argsort(p_val)
ranked = p_val[order]
n      = ranked.size
q_tmp  = ranked * n / (np.arange(n) + 1)
q_adj  = np.minimum.accumulate(q_tmp[::-1])[::-1]
q_val  = np.empty_like(q_adj)
q_val[order] = q_adj

dmp = pd.DataFrame({"diff": diff.values, "p": p_val, "q": q_val},
                   index=methyl_use.index).sort_values("q")

sig_mask = (dmp["q"] < fdr_thr) & (dmp["diff"].abs() >= diff_thr)
print(f"유의한 DMP: {sig_mask.sum()}개 (FDR<{fdr_thr}, |Δβ|≥{diff_thr})")

In [ ]:
# Volcano plot
x = dmp["diff"].to_numpy()
y = -np.log10(np.clip(dmp["p"].to_numpy(), 1e-300, 1.0))

plt.figure(figsize=(10, 7))
plt.scatter(x, y, s=8, alpha=0.6, c='lightblue', edgecolors='none')
plt.axhline(-np.log10(fdr_thr), ls="--", c="red", alpha=0.7, lw=2, label=f"FDR {fdr_thr}")
plt.axvline(+diff_thr, ls="--", c="green", alpha=0.7, lw=2)
plt.axvline(-diff_thr, ls="--", c="green", alpha=0.7, lw=2, label=f"|Δβ| {diff_thr}")
plt.scatter(x[sig_mask], y[sig_mask], s=15, alpha=0.8, c='red', edgecolors='darkred', label='Significant')

plt.title(f"DMP Volcano Plot: {subtype_a} vs {subtype_b}")
plt.xlabel("Mean β difference (A - B)")
plt.ylabel("-log₁₀(p-value)")
plt.legend(); plt.grid(True, alpha=0.3)
plt.tight_layout(); plt.show()

# Top 5 요약
print(f"\nTop 5 Hypermethylated:")
print(dmp[dmp["diff"] > 0].head(5)[["diff","q"]].to_string())
print(f"\nTop 5 Hypomethylated:")
print(dmp[dmp["diff"] < 0].head(5)[["diff","q"]].to_string())

### Vibe Coding 미션 B3: DMP 분석 심화
**미션**: DMP 결과에서 상위 유의한 프로브들의 메틸화 패턴을 히트맵으로 시각화해보세요!

**결과물**: Top 20 DMP의 아형별 메틸화 히트맵 (그룹 컬러바 포함)

**도전 과제**:
1. 상위 **20개 유의한 DMP** 선별하기
2. 샘플을 Subtype별로 정렬하여 **히트맵** 생성하기  
3. (도전) 히트맵에 그룹 구분을 위한 **컬러바** 추가하기

In [ ]:
# 미션 B3 코드를 여기에 작성하세요!
